# Fit the Model to Training Data

## Setup

In [2]:
from importlib import reload

import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt

from hot_dust import preprocess, model

In [2]:
# "reload" to get changes in preprocess.py without restarting the kernel
reload(preprocess)
reload(model)
from hot_dust.preprocess import prepare_training_data, split_training_data
from hot_dust.model import to_tensorflow, compile, pretraining

### Parameters

In [3]:
epochs = 200  # Max 500
batch_size = 64
buffer_size = 10 * batch_size

ds = prepare_training_data()
train, validate, test = to_tensorflow(split_training_data(ds))

train = train.shuffle(buffer_size).batch(batch_size)
validate = validate.batch(batch_size).cache()

In [4]:
layer = pretraining(dataset=test)
network = compile(normalization=layer)
network.summary()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 normalization (Normalizatio  (None, 9)                19        
 n)                                                              
                                                                 
 dense (Dense)               (None, 16)                160       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17 

### Fitting

In [5]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=20,
    verbose=1,
)
# List for callbacks
callbacks_list = [early_stopping]

fit = network.fit(
    train,
    epochs=epochs,
    validation_data=validate,
    verbose=2,
    callbacks=[early_stopping],
)

network.save("data/network")
fit = {"epoch": fit.epoch, **fit.history}
np.savez("data/fit.npz", **fit)

Epoch 1/200
3122/3122 - 4s - loss: 0.6225 - val_loss: 0.3767 - 4s/epoch - 1ms/step
Epoch 2/200
3122/3122 - 3s - loss: 0.3117 - val_loss: 0.2749 - 3s/epoch - 872us/step
Epoch 3/200
3122/3122 - 3s - loss: 0.2486 - val_loss: 0.2288 - 3s/epoch - 908us/step
Epoch 4/200
3122/3122 - 9s - loss: 0.2100 - val_loss: 0.1960 - 9s/epoch - 3ms/step
Epoch 5/200
3122/3122 - 6s - loss: 0.1844 - val_loss: 0.1741 - 6s/epoch - 2ms/step
Epoch 6/200
3122/3122 - 3s - loss: 0.1673 - val_loss: 0.1599 - 3s/epoch - 833us/step
Epoch 7/200
3122/3122 - 3s - loss: 0.1544 - val_loss: 0.1485 - 3s/epoch - 881us/step
Epoch 8/200
3122/3122 - 3s - loss: 0.1444 - val_loss: 0.1414 - 3s/epoch - 811us/step
Epoch 9/200
3122/3122 - 3s - loss: 0.1357 - val_loss: 0.1316 - 3s/epoch - 832us/step
Epoch 10/200
3122/3122 - 4s - loss: 0.1287 - val_loss: 0.1265 - 4s/epoch - 1ms/step
Epoch 11/200
3122/3122 - 3s - loss: 0.1226 - val_loss: 0.1185 - 3s/epoch - 818us/step
Epoch 12/200
3122/3122 - 3s - loss: 0.1173 - val_loss: 0.1147 - 3s/epoc

INFO:tensorflow:Assets written to: data/network\assets


INFO:tensorflow:Assets written to: data/network\assets


### Training and Validation Losses

In [3]:
fit = np.load("data/fit.npz")

In [4]:
model.plot_loss(fit)

:NdOverlay   [split]
   :Scatter   [epoch]   (value)